In [73]:
import pandas as pd
import numpy as np

In [74]:
# Funcion para cargar el archivo excel
def cargar_archivo_excel(archivo, hojas, engine='openpyxl'):
    try:
        xls_file = pd.ExcelFile(archivo, engine=engine)
        dfs = {}

        for hoja in hojas:
            df = pd.read_excel(xls_file, hoja) 
            dfs[hoja] = df

        return dfs

    except Exception as e:
        print(f"Error al cargar el archivo Excel: {e}")
        return None

In [75]:
# Se carga el archivo
ruta="homicidios.xlsx"
pags=["HECHOS","VICTIMAS"]
datos_excel=cargar_archivo_excel(ruta,pags)

In [76]:
df_datos_hechos=datos_excel["HECHOS"]

In [77]:
df_datos_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [78]:
# Se obtiene la informacion general del dataframe
df_datos_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [79]:
# Se convierten las columnas a minuscula para evitar problemas
columnas_a_minusculas = ['TIPO_DE_CALLE', 'VICTIMA','ACUSADO']

# Se itera sobre las columnas seleccionadas y se pasan a minúsculas
for columna in columnas_a_minusculas:
    df_datos_hechos[columna] = df_datos_hechos[columna].str.lower()

In [80]:
df_datos_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [81]:
# Se eliminan las columnas ("aaaa","mm","dd") ya que hay una columna fecha que abarca la misma informacion
# Se elimina las columnas ("lugar_del_hecho",'Calle','Altura','Cruce') ya que la columna direccion normalizada resume la misma informacion
# Se elimina las columnas ('xy (caba)','participantes') ya que ambas columnas contienen datos que estan en otras columnas del dataframe
columnas_eliminar = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 'Altura',
                     'Cruce', 'XY (CABA)', 'PARTICIPANTES', 'Dirección Normalizada']


df_datos_hechos.drop(columnas_eliminar,axis=1,inplace=True)

In [82]:
df_datos_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             696 non-null    object        
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         696 non-null    int64         
 6   pos x          696 non-null    object        
 7   pos y          696 non-null    object        
 8   VICTIMA        696 non-null    object        
 9   ACUSADO        696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 54.5+ KB


In [83]:
def analizar_valores_sd(dataframe):
    columnas_con_sd = dataframe.columns
    resultados = []

    for columna in columnas_con_sd:
        cantidad_sd = dataframe[columna].eq('SD').sum()
        resultados.append({'Columna': columna, 'Cantidad de SD': cantidad_sd})

    resultados_df = pd.DataFrame(resultados)
    resultados_con_sd = resultados_df[resultados_df['Cantidad de SD'] > 0]

    return resultados_con_sd

In [84]:
# Se cuentan la cantidad de valores "SD" o "sd" (sin datos) que hay en todo el dataframe
analizar_valores_sd(df_datos_hechos)

,Columna,Cantidad de SD
3,HH,1


In [85]:
# Se remplazan los valores "SD" o "sd" por NaN en todo el dataframe
df_datos_hechos.replace(['SD','sd'], np.nan, inplace=True)

In [86]:
# Verificamos que se hayan efectuado los cambios
analizar_valores_sd(df_datos_hechos)

,Columna,Cantidad de SD


In [87]:
# Filtrar las filas donde la columna 'hh' tiene valores NaN
filas_con_nan = df_datos_hechos[df_datos_hechos['HH'].isna()]
print(filas_con_nan)

            ID  N_VICTIMAS      FECHA  HH TIPO_DE_CALLE  COMUNA         pos x  \
518  2019-0103           1 2019-12-18 NaN      gral paz      11  -58.52169422   

            pos y VICTIMA ACUSADO  
518  -34.59471640    moto    moto  


In [88]:
# Se convierte la columna 'hh' a números, manejar errores y convertir no numéricos a NaN
df_datos_hechos['HH'] = pd.to_numeric(df_datos_hechos['HH'], errors='coerce')

# Se convierte a 'float64' para admitir NaN
df_datos_hechos['HH'] = df_datos_hechos['HH'].astype('float64')

# Se convierte a 'Int64' para preservar NaN y tratar los demás valores como enteros
df_datos_hechos['HH'] = df_datos_hechos['HH'].astype('Int64')

In [89]:
# Lista de columnas que se van a categorizar. Se categorizan las columnas con el objetivo de 
# facilitar el analisis exploratorio y las visualizaciones.
columns_to_categorize = ['HH', 'TIPO_DE_CALLE', 'COMUNA', 'VICTIMA', 'ACUSADO']

for column in columns_to_categorize:
    df_datos_hechos[column] = pd.Categorical(df_datos_hechos[column])


In [90]:
# Lista de columnas a verificar
columnas_a_verificar = ['COMUNA', 'pos x', 'pos y']

# Se filtran las filas con valores iguales a 0 o "." en las columnas especificadas
filas_con_cero_o_vacios = df_datos_hechos[df_datos_hechos[columnas_a_verificar].eq(0).any(axis=1) | df_datos_hechos[columnas_a_verificar].eq('.').any(axis=1)]

# Mostrar las filas seleccionadas
print(filas_con_cero_o_vacios)


            ID  N_VICTIMAS      FECHA  HH TIPO_DE_CALLE COMUNA pos x pos y  \
38   2016-0052           1 2016-04-20  20     autopista     13     .     .   
106  2016-0136           1 2016-10-25   0     autopista      4     .     .   
119  2016-0151           1 2016-11-18  20         calle      0     .     .   
139  2016-0174           1 2016-12-27   0     autopista      0     .     .   
176  2017-0042           1 2017-04-10   9      gral paz     14     .     .   
180  2017-0050           2 2017-04-28  11     autopista      9     .     .   
181  2017-0051           1 2017-05-01   3     autopista      7     .     .   
256  2017-0140           1 2017-11-19  23     autopista      4     .     .   
313  2018-0039           1 2018-04-21  22     autopista     14     .     .   
546  2020-0026           1 2020-05-17   6     autopista     14     .     .   
559  2020-0039           1 2020-09-01  19         calle      9     .     .   
621  2021-0023           1 2021-03-01   9     autopista      4  

In [91]:
registros_vacios=[
    {'ID': '2016-0052', 'COMUNA': 13, 'pos x':  -58.446910738, 'pos y': -34.543916034}, 
    {'ID': '2016-0136', 'COMUNA': 4, 'pos x': -58.355021188, 'pos y': -34.629822714},  
    {'ID': '2016-0151', 'COMUNA': 10, 'pos x': -58.52756365, 'pos y': -34.62772274}, 
    {'ID': '2016-0174', 'COMUNA': 7, 'pos x': -58.439199691, 'pos y': -34.631119348}, 
    {'ID': '2017-0042', 'COMUNA': 14, 'pos x': -58.457579360, 'pos y': -34.538957930}, 
    {'ID': '2017-0050', 'COMUNA': 9, 'pos x': -58.481800300, 'pos y': -34.647005340}, 
    {'ID': '2017-0051', 'COMUNA': 7, 'pos x':  -58.49389816, 'pos y': -34.63965969},      
    {'ID': '2017-0140', 'COMUNA': 4, 'pos x': -58.37676334, 'pos y': -34.637748009}, 
    {'ID': '2018-0039', 'COMUNA': 14, 'pos x': -58.438563633, 'pos y': -34.548451872},   
    {'ID': '2020-0026', 'COMUNA': 14, 'pos x': -34.54696676, 'pos y': -58.439152924}, 
    {'ID': '2020-0039', 'COMUNA': 9, 'pos x':  -58.493233104, 'pos y': -34.663248927},    
    {'ID': '2021-0023', 'COMUNA': 4, 'pos x': -57.95949970, 'pos y': -34.927029412},   
] 


# Se itera sobre los registros y realizar el reemplazo para todas las filas
for registro in registros_vacios:
    id = registro['ID']
    comuna = registro['COMUNA']
    pos_x = registro['pos x']
    pos_y= registro['pos y']

    # Se utiliza loc para reemplazar los valores específicos para todas las filas
    df_datos_hechos.loc[df_datos_hechos['ID'] == id, 'COMUNA'] = comuna
    df_datos_hechos.loc[df_datos_hechos['ID'] == id, 'pos x'] = pos_x
    df_datos_hechos.loc[df_datos_hechos['ID'] == id, 'pos y'] = pos_y


Se remplazan los valores vacios dentro de 'comuna', 'pos_x' y 'pos_y'.La mayoria pudieron ser ubicados en google maps gracias a las calles, solo hubo uno(2016-0151) que no tenia comuna, calle, latitud ni longitud, solo tenia el tipo de calle y se procedio a tomar los valores del siniestro mas proximo, que se haya dado en una calle para completar los datos faltantes. 

In [92]:
# Lista de columnas a verificar
columnas_a_verificar = ['COMUNA', 'pos x', 'pos y']

# Se filtran las filas con valores iguales a 0 o "." en las columnas especificadas
filas_con_cero_o_vacios = df_datos_hechos[df_datos_hechos[columnas_a_verificar].eq(0).any(axis=1) | df_datos_hechos[columnas_a_verificar].eq('.').any(axis=1)]

# Mostrar las filas seleccionadas
print(filas_con_cero_o_vacios)


Empty DataFrame
Columns: [ID, N_VICTIMAS, FECHA, HH, TIPO_DE_CALLE, COMUNA, pos x, pos y, VICTIMA, ACUSADO]
Index: []


In [93]:
# Se convierte las columnas "pos_x" y "pos_y" a tipo de dato float
df_datos_hechos['pos x'] = pd.to_numeric(df_datos_hechos['pos x'], errors='coerce').astype(float)
df_datos_hechos['pos y'] = pd.to_numeric(df_datos_hechos['pos y'], errors='coerce').astype(float)


In [94]:
# Se crea un diccionario con los nuevos nombres de las columnas
mapeo_nombres = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas','HH':'franja_horaria',
                'pos x':'longitud','pos y':'latitud','VICTIMA':'vehiculo_victima',
                'ACUSADO':'vehiculo_acusado','FECHA':'fecha','TIPO_DE_CALLE':'tipo_de_calle','COMUNA':'comuna'}

# Utiliza el método rename para renombrar las columnas específicas
df_datos_hechos.rename(columns=mapeo_nombres, inplace=True)


In [95]:
print("\nETL DataFrame:")
df_datos_hechos.head()


ETL DataFrame:


,id_siniestro,nro_victimas,fecha,franja_horaria,tipo_de_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.475340,-34.687570,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.508775,-34.669777,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.390403,-34.631894,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.465039,-34.680930,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.387183,-34.622466,moto,pasajeros


DATASETS HOMICIDIOS VICTIMAS

In [96]:
df_victimas=datos_excel['VICTIMAS']

In [97]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [98]:
# Se obtiene la informacion general del dataframe
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [99]:
# Invoca la función 'analizar_valores_sd'

resultados_victimas = analizar_valores_sd(df_victimas)
resultados_victimas

,Columna,Cantidad de SD
5,ROL,11
6,VICTIMA,9
7,SEXO,6
8,EDAD,53
9,FECHA_FALLECIMIENTO,68


In [100]:
# Se reemplaza 'SD' por NaN en todo el DataFrame

df_victimas.replace(['SD','sd'], np.nan, inplace=True)

In [101]:
columnas_eliminar=['FECHA','AAAA','MM','DD','VICTIMA','FECHA_FALLECIMIENTO']
df_victimas.drop(columnas_eliminar,axis=1,inplace=True)

In [102]:
# Se convierte la columna 'EDAD' a float y despues a int para preservar Nan
df_victimas['EDAD'] = pd.to_numeric(df_victimas['EDAD'], errors='coerce')

# Se convierte a 'float64' para admitir NaN
df_victimas['EDAD'] = df_victimas['EDAD'].astype('float64')

# Se convierte a 'Int64' para preservar NaN y tratar los demás valores como enteros
df_victimas['EDAD'] = df_victimas['EDAD'].astype('Int64')

In [103]:
# Se convierten las columnas "SEXO" y "ROL" a minuscula para evitar problemas
columnas_a_minusculas = ['SEXO', 'ROL']

# Se itera sobre las columnas seleccionadas y se pasan a minúsculas
for columna in columnas_a_minusculas:
    df_victimas[columna] = df_victimas[columna].str.lower()

In [104]:
df_victimas.head()

,ID_hecho,ROL,SEXO,EDAD
0,2016-0001,conductor,masculino,19
1,2016-0002,conductor,masculino,70
2,2016-0003,conductor,masculino,30
3,2016-0004,conductor,masculino,18
4,2016-0005,conductor,masculino,29


In [105]:
# Lista de columnas que se van a categorizar. Se categorizan las columnas con el objetivo de 
# facilitar el analisis exploratorio y las visualizaciones.
columns_to_categorize = ['ROL', 'SEXO']

for column in columns_to_categorize:
    df_victimas[column] = pd.Categorical(df_victimas[column])

In [106]:
# Se crea un diccionario con los nuevos nombres de las columnas
mapeo_nombres = {'ID_hecho':'id_siniestro','ROL':'rol','SEXO':'sexo','EDAD':'edad'}

# Utiliza el método rename para renombrar las columnas específicas
df_victimas.rename(columns=mapeo_nombres, inplace=True)

In [107]:
df_victimas

,id_siniestro,rol,sexo,edad
0,2016-0001,conductor,masculino,19
1,2016-0002,conductor,masculino,70
2,2016-0003,conductor,masculino,30
3,2016-0004,conductor,masculino,18
4,2016-0005,conductor,masculino,29
...,...,...,...,...
712,2021-0092,peaton,femenino,50
713,2021-0093,pasajero_acompañante,femenino,18
714,2021-0094,pasajero_acompañante,femenino,43
715,2021-0095,conductor,masculino,27


DATASET LESIONES HECHOS

In [108]:
lesiones='lesiones.xlsx'
pags='HECHOS','VICTIMAS'
datos_Excel_2=cargar_archivo_excel(lesiones, pags)

In [109]:
df_hechos_lesiones= datos_Excel_2['HECHOS']

In [110]:
df_hechos_lesiones.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [111]:
# Se Obtiene la información general del DataFrame
df_hechos_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [112]:
columnas_eliminar=['aaaa','mm', 'dd', 'hora', 'direccion_normalizada', 'otra_direccion', 'calle', 'altura', 'cruce',
                   'participantes', 'geocodificacion_CABA', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista']

df_hechos_lesiones.drop(columnas_eliminar,axis=1,inplace=True)

In [113]:
sin_datos=analizar_valores_sd(df_hechos_lesiones)
sin_datos

,Columna,Cantidad de SD
4,comuna,846
5,tipo_calle,11045
6,longitud,1209
7,latutid,1209
8,victima,10733
9,acusado,15288
10,gravedad,23056


In [114]:
# Se reemplaza 'SD' por NaN en todo el DataFrame

df_hechos_lesiones.replace(['SD','sd'], np.nan, inplace=True)

In [115]:
sin_datos=analizar_valores_sd(df_hechos_lesiones)
sin_datos

,Columna,Cantidad de SD


In [116]:
df_hechos_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           23785 non-null  object 
 1   n_victimas   23785 non-null  int64  
 2   fecha        23785 non-null  object 
 3   franja_hora  23776 non-null  float64
 4   comuna       22770 non-null  object 
 5   tipo_calle   12740 non-null  object 
 6   longitud     22314 non-null  object 
 7   latutid      22314 non-null  object 
 8   victima      13052 non-null  object 
 9   acusado      8497 non-null   object 
 10  gravedad     646 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 2.0+ MB


In [117]:
# Se convierten las columnas a minuscula para evitar problemas
columnas_a_minusculas = ['tipo_calle', 'victima','acusado','gravedad']

# Se itera sobre las columnas seleccionadas y se pasan a minúsculas
for columna in columnas_a_minusculas:
    df_hechos_lesiones[columna] = df_hechos_lesiones[columna].str.lower()

In [118]:
# Se reemplazan valores nulos en 'gravedad'
df_hechos_lesiones['gravedad'] = df_hechos_lesiones['gravedad'].fillna('leve')

Se realiza el remplazo de valores nulos por 'leve', ya que en el diccionario de datos dice lo siguiente: " A efectos analíticos, los casos sin datos se corresponden con una alta probabilidad a casos leves."

In [119]:
# Se reemplazan valores nulos en 'comuna'
df_hechos_lesiones['comuna'] = df_hechos_lesiones['comuna'].replace('No Especificada', np.nan)

In [120]:
# Se convierte las columnas "latutid" y "longitud" a tipo de dato float
df_hechos_lesiones['latutid'] = pd.to_numeric(df_hechos_lesiones['latutid'], errors='coerce').astype(float)
df_hechos_lesiones['longitud'] = pd.to_numeric(df_hechos_lesiones['longitud'], errors='coerce').astype(float)

In [121]:
columns_to_categorize = ['franja_hora', 'tipo_calle','comuna','victima','acusado','gravedad']

for column in columns_to_categorize:
    df_hechos_lesiones[column] = pd.Categorical(df_hechos_lesiones[column])

In [122]:
df_hechos_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           23785 non-null  object  
 1   n_victimas   23785 non-null  int64   
 2   fecha        23785 non-null  object  
 3   franja_hora  23776 non-null  category
 4   comuna       22328 non-null  category
 5   tipo_calle   12740 non-null  category
 6   longitud     22314 non-null  float64 
 7   latutid      22314 non-null  float64 
 8   victima      13052 non-null  category
 9   acusado      8497 non-null   category
 10  gravedad     23785 non-null  category
dtypes: category(6), float64(2), int64(1), object(2)
memory usage: 1.0+ MB


In [123]:
# Se crea un diccionario con los nuevos nombres de las columnas
mapeo_nombres = {'id': 'id_siniestro', 'n_victimas': 'nro_victimas','franja_hora':'franja_horaria',
                'latutid':'latitud','victima':'vehiculo_victima',
                'acusado':'vehiculo_acusado'}

# Se utiliza el método rename para renombrar las columnas específicas
df_hechos_lesiones.rename(columns=mapeo_nombres, inplace=True)

In [124]:
df_hechos_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_siniestro      23785 non-null  object  
 1   nro_victimas      23785 non-null  int64   
 2   fecha             23785 non-null  object  
 3   franja_horaria    23776 non-null  category
 4   comuna            22328 non-null  category
 5   tipo_calle        12740 non-null  category
 6   longitud          22314 non-null  float64 
 7   latitud           22314 non-null  float64 
 8   vehiculo_victima  13052 non-null  category
 9   vehiculo_acusado  8497 non-null   category
 10  gravedad          23785 non-null  category
dtypes: category(6), float64(2), int64(1), object(2)
memory usage: 1.0+ MB


DATASET LESIONES VICTIMA

In [125]:
df_lesiones_victimas=datos_Excel_2['VICTIMAS']

In [126]:
df_lesiones_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [127]:
# Se obtiene información general del DataFrame

df_lesiones_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


In [128]:
columnas_eliminar=['FECHA ','AAA','MM','DD','VEHICULO_VICTIMA','GRAVEDAD']
df_lesiones_victimas.drop(columnas_eliminar,axis=1,inplace=True)

In [129]:
# Se utiliza la funcion analizar_valores_sd para hacer un conteo por columna de los valores sd o SD
pasar_nulos=analizar_valores_sd(df_lesiones_victimas)
pasar_nulos

,Columna,Cantidad de SD
1,SEXO,1929
2,EDAD_VICTIMA,3990


In [130]:
# Se reemplaza 'SD' por NaN en todo el DataFrame

df_lesiones_victimas.replace(['SD','sd'], np.nan, inplace=True)

In [131]:
# Se utiliza la funcion analizar_valores_sd para hacer un conteo por columna de los valores sd o SD
pasar_nulos=analizar_valores_sd(df_lesiones_victimas)
pasar_nulos

,Columna,Cantidad de SD


In [132]:
# Se convierten las columnas a minuscula para evitar problemas
df_lesiones_victimas['SEXO'] = df_lesiones_victimas['SEXO'].str.upper()



In [133]:
df_lesiones_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID hecho      27605 non-null  object
 1   SEXO          25377 non-null  object
 2   EDAD_VICTIMA  23134 non-null  object
dtypes: object(3)
memory usage: 647.1+ KB


In [134]:
# Se crea un diccionario con los valores a remplazar
values_to_replace = {'SEXO': {'VARON': 'masculino', 'MUJER': 'femenino','MUJER ':'femenino'}}

# Se utiliza el método replace para realizar el reemplazo
df_lesiones_victimas.replace(values_to_replace, inplace=True)

In [135]:

columns_to_categorize = ['SEXO']

for column in columns_to_categorize:
    df_lesiones_victimas[column] = pd.Categorical(df_lesiones_victimas[column])

In [136]:

# Verificar los valores únicos en la columna 'sexo'
print(df_lesiones_victimas['SEXO'].unique())

['masculino', NaN, 'femenino']
Categories (2, object): ['femenino', 'masculino']


In [137]:
# Se convierte la columna 'EDAD' a float y despues a int para preservar Nan
df_lesiones_victimas['EDAD_VICTIMA'] = pd.to_numeric(df_lesiones_victimas['EDAD_VICTIMA'], errors='coerce')

# Se convierte a 'float64' para admitir NaN
df_lesiones_victimas['EDAD_VICTIMA'] = df_lesiones_victimas['EDAD_VICTIMA'].astype('float64')

# Se convierte a 'Int64' para preservar NaN y tratar los demás valores como enteros
df_lesiones_victimas['EDAD_VICTIMA'] = df_lesiones_victimas['EDAD_VICTIMA'].astype('Int64')

In [138]:
# Se crea un diccionario con los nuevos nombres de las columnas
mapeo_nombres = {'ID hecho': 'id_siniestro', 'SEXO':'sexo','EDAD_VICTIMA':'edad'}

# Se utiliza el método rename para renombrar las columnas específicas
df_lesiones_victimas.rename(columns=mapeo_nombres, inplace=True)

In [139]:
df_lesiones_victimas

,id_siniestro,sexo,edad
0,LC-2019-0000053,masculino,57
1,LC-2019-0000063,NaN,<NA>
2,LC-2019-0000079,masculino,<NA>
3,LC-2019-0000082,masculino,45
4,LC-2019-0000082,femenino,45
...,...,...,...
27600,LC-2021-0451911,masculino,87
27601,LC-2021-0530228,femenino,60
27602,LC-2021-0530228,femenino,32
27603,LC-2021-0201378,masculino,32


COMBINAR DATAFRAMES

In [140]:
# Se fusionan los DataFrames

df_homicidios = pd.merge(df_datos_hechos, df_victimas, left_on='id_siniestro', right_on='id_siniestro', how='inner')
df_lesiones = pd.merge(df_hechos_lesiones, df_lesiones_victimas, left_on='id_siniestro', right_on='id_siniestro', how='inner')

In [141]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_siniestro      717 non-null    object        
 1   nro_victimas      717 non-null    int64         
 2   fecha             717 non-null    datetime64[ns]
 3   franja_horaria    716 non-null    category      
 4   tipo_de_calle     717 non-null    category      
 5   comuna            717 non-null    category      
 6   longitud          717 non-null    float64       
 7   latitud           717 non-null    float64       
 8   vehiculo_victima  708 non-null    category      
 9   vehiculo_acusado  694 non-null    category      
 10  rol               706 non-null    category      
 11  sexo              711 non-null    category      
 12  edad              664 non-null    Int64         
dtypes: Int64(1), category(7), datetime64[ns](1), float64(2), int64(1), object(1)
mem

In [142]:
df_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_siniestro      27598 non-null  object  
 1   nro_victimas      27598 non-null  int64   
 2   fecha             27598 non-null  object  
 3   franja_horaria    27590 non-null  category
 4   comuna            25974 non-null  category
 5   tipo_calle        14719 non-null  category
 6   longitud          25942 non-null  float64 
 7   latitud           25942 non-null  float64 
 8   vehiculo_victima  14776 non-null  category
 9   vehiculo_acusado  9680 non-null   category
 10  gravedad          27598 non-null  category
 11  sexo              25370 non-null  category
 12  edad              23127 non-null  Int64   
dtypes: Int64(1), category(7), float64(2), int64(1), object(2)
memory usage: 1.5+ MB


In [143]:
# Se reorganizan columnas de df_lesiones

reorden = ['id_siniestro', 'nro_victimas', 'fecha', 'franja_horaria', 'tipo_calle', 'comuna', 'longitud', 'latitud', 
           'vehiculo_victima', 'vehiculo_acusado', 'sexo', 'edad', 'gravedad']

df_lesiones = df_lesiones[reorden]

In [144]:
# Se almacenan los DataFrames finales en un nuevo archivo CSV

df_homicidios.to_csv('homicidios_limpio.csv', index=False)
df_lesiones.to_csv('lesiones_limpio.csv', index=False)